In [3]:
# -*- coding:utf-8 -*-

In [4]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import chart_studio.plotly as py
import cufflinks as cf
import plotly.offline as plyo
import peakutils

In [5]:
import chart_studio
import plotly.graph_objects as go

chart_studio.tools.set_credentials_file(username='dorosi', api_key='L2x3IMvhlfHqaeBCfV0z')

## ED extraction

In [210]:
def peakfinding_ED(group, num):
    
    x_axis = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/smoothing_xarray/'+num+'_smoothing_xarray.npy')    
    y_axis = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/smoothing_yarray/'+num+'_smoothing_yarray.npy')
    
    x_arr = np.array(x_axis)
    y_arr = np.array(y_axis)
    
    index = peakutils.indexes(120-y_arr, thres=0.8, min_dist=20)
    
    peak = y_arr[index]
    index = x_arr[index]
    
    np.save('./Ultrasound Imaging Processing_real_update/'+group+'/EDvelocity_xarray/%s_EDvelocity_xarray' %num, index)
    np.save('./Ultrasound Imaging Processing_real_update/'+group+'/EDvelocity_yarray/%s_EDvelocity_yarray' %num, peak)
        
    #plt.figure(figsize = (20, 6))
    #plt.plot(x_axis, y_axis, color="#2ECC71")
    #plt.gca().invert_yaxis()
    #plt.plot(index, peak, 'o', color='m')
    #plt.title('Heart rate over Time')
    #plt.xlabel('time(sec)')
    #plt.ylabel('speed(cm/s)')
    
    file_path = "./Ultrasound Imaging Processing_real_update/"+group+"/EDvelocity_graph/"+num+"_EDvelocity_graph.html"

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_axis, y=y_axis, name='original', line=dict(color='#2ECC71', width=1)))
    fig.add_trace(go.Scatter(x=index, y=peak, name='based on peak', mode='markers'))
    fig.update_layout(xaxis = dict(tickmode = 'linear', tick0 = 0, dtick = 20))
    fig.write_html(file_path)

In [193]:
path = './Ultrasound Imaging Processing_real_update/Normal/smoothing_xarray/'
file_list = os.listdir(path)
file_list.sort()
file_list = file_list[9:10]
file_list

['14_smoothing_xarray.npy']

In [211]:
# 하나씩 조정

path = './Ultrasound Imaging Processing_real_update/Normal/smoothing_xarray/'
file_list = os.listdir(path)
file_list.sort()
file_list = file_list[9:10]

for i in range(len(file_list)):
    
    y_axis = path + "/" + file_list[i]
    
    num = file_list[i][:2]
    
    peakfinding_ED("Normal", num)
    
    print(num)

14


## ED filtering

In [204]:
def filtering_EDvelocity(group, num):

    x_axis = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/EDvelocity_xarray/'+num+'_EDvelocity_xarray.npy')
    y_axis = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/EDvelocity_yarray/'+num+'_EDvelocity_yarray.npy')
    
    x = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/smoothing_xarray/'+num+'_smoothing_xarray.npy')    
    y = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/smoothing_yarray/'+num+'_smoothing_yarray.npy')
    
    for i in range(len(y_axis)):
    
        if i < (len(y_axis)-1) :
            if abs(x_axis[i]-x_axis[i+1]) < 0.6 :
                if y_axis[i] > y_axis[i+1] :
                    # 0으로 변환
                    x_axis[i] = 0
                    y_axis[i] = 0
            
                else:
                    x_axis[i+1] = 0
                    y_axis[i+1] = 0
    
    new_x = []
    new_y = []
    
    # 0인 값만 제외하고 x,y 저장
    for i in range(len(y_axis)):
    
        if y_axis[i] != 0:
            new_y.append(y_axis[i])
            new_x.append(x_axis[i])
    
    np.save('./Ultrasound Imaging Processing_real_update/'+group+'/EDvelocity_xarray/%s_EDvelocity_xarray' %num, new_x)
    np.save('./Ultrasound Imaging Processing_real_update/'+group+'/EDvelocity_yarray/%s_EDvelocity_yarray' %num, new_y)
        
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, name='original', line=dict(color='#2ECC71', width=1)))
    fig.add_trace(go.Scatter(x=new_x, y=new_y, name='based on peak', mode='markers'))
    fig.update_layout(xaxis = dict(tickmode = 'linear', tick0 = 0, dtick = 20))
    fig.write_html("./Ultrasound Imaging Processing_real_update/"+group+"/EDvelocity_graph/"+num+"_EDvelocity_graph.html")      

In [212]:
# 하나씩 조정

path = './Ultrasound Imaging Processing_real_update/Normal/smoothing_yarray/'
file_list = os.listdir(path)
file_list.sort()
file_list = file_list[9:10]

for i in range(len(file_list)):
    
    y_axis = path + "/" + file_list[i]
    
    num = file_list[i][:2]
    
    filtering_EDvelocity("Normal", num)

## PS extraction

In [206]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx


def find_ps_through_ed(group, num):
    
    ed_x_axis = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/EDvelocity_xarray/'+num+'_EDvelocity_xarray.npy')
    ed_y_axis = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/EDvelocity_yarray/'+num+'_EDvelocity_yarray.npy')
    
    x_axis = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/smoothing_xarray/'+num+'_smoothing_xarray.npy')
    y_axis = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/smoothing_yarray/'+num+'_smoothing_yarray.npy')
    
    li_x_axis = x_axis.tolist()
    li_y_axis = y_axis.tolist()
    
    round_index = []

    for i in range(len(ed_x_axis)):
        round_index.append(find_nearest(x_axis, ed_x_axis[i]))           # ed 인덱스 값
        
    ps_x_axis = []   
    ps_y_axis = []

    for i in range(len(round_index)-1):
        ps_range = y_axis[round_index[i]:round_index[i+1]]
        ps = ps_range.max()
        ps_x = li_y_axis.index(ps)
        #print(ps)
        ps_y_axis.append(ps)
        ps_x_axis.append(li_x_axis[ps_x])
        
        
    np.save('./Ultrasound Imaging Processing_real_update/'+group+'/PSvelocity_xarray/%s_PSvelocity_xarray' %num, ps_x_axis)
    np.save('./Ultrasound Imaging Processing_real_update/'+group+'/PSvelocity_yarray/%s_PSvelocity_yarray' %num, ps_y_axis)    
        
        
    # ED / PS 그래프    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_axis, y=y_axis, name='smoothing', line=dict(color='#2ECC71', width=1)))
    fig.add_trace(go.Scatter(x=ps_x_axis, y=ps_y_axis, name='PS', mode='markers', line=dict(color='#FF6347')))
    #fig.add_trace(go.Scatter(x=ed_x_axis, y=ed_y_axis, name='ED', mode='markers', line=dict(color='#8282FF')))
    fig.update_layout(xaxis = dict(tickmode = 'linear', tick0 = 0, dtick = 20))
    
    file_name = "./Ultrasound Imaging Processing_real_update/"+group+"/PSvelocity_graph/"+num+"_PSvelocity_graph.html"
    fig.write_html('%s' %file_name)
    
    #print(len(ps_y_axis))

In [180]:
path = './Ultrasound Imaging Processing_real_update/Normal/smoothing_yarray/'
file_list = os.listdir(path)
file_list.sort()
file_list[9:10]


['14_smoothing_yarray.npy']

In [213]:
path = './Ultrasound Imaging Processing_real_update/Normal/smoothing_yarray/'
file_list = os.listdir(path)
file_list.sort()
file_list = file_list[9:10]

for i in range(len(file_list)):
    
    y_axis = path + "/" + file_list[i]
    
    num = file_list[i][:2]
    
    find_ps_through_ed("Normal", num)

In [208]:
def PSnED_graph(group, num):
    
    x_axis = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/smoothing_xarray/'+num+'_smoothing_xarray.npy')
    y_axis = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/smoothing_yarray/'+num+'_smoothing_yarray.npy')
    
    PS_x = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/PSvelocity_xarray/'+num+'_PSvelocity_xarray.npy')
    PS_y = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/PSvelocity_yarray/'+num+'_PSvelocity_yarray.npy')
    ED_x = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/EDvelocity_xarray/'+num+'_EDvelocity_xarray.npy')
    ED_y = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/EDvelocity_yarray/'+num+'_EDvelocity_yarray.npy')
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_axis, y=y_axis, name='original', line=dict(color='#2ECC71', width=1)))
    fig.add_trace(go.Scatter(x=PS_x, y=PS_y, name='PS', mode='markers', line=dict(color='#FF6347')))
    fig.add_trace(go.Scatter(x=ED_x, y=ED_y, name='ED', mode='markers', line=dict(color='#8282FF')))
    fig.update_layout(xaxis = dict(tickmode = 'linear', tick0 = 0, dtick = 20))
    
    file_name = "./Ultrasound Imaging Processing_real_update/"+group+"/PSnEDvelocity_graph/"+num+"_PSnEDvelocity_graph.html"
    fig.write_html('%s' %file_name)
    
    print("PS velocity 개수 : ", len(PS_x))
    print("ED velocity 개수 : ", len(ED_x))

In [200]:
path = './Ultrasound Imaging Processing_real_update/Normal/smoothing_yarray/'
file_list = os.listdir(path)
file_list.sort()
file_list = file_list[9:10]
file_list

['14_smoothing_yarray.npy']

In [214]:
path = './Ultrasound Imaging Processing_real_update/Normal/smoothing_yarray/'
file_list = os.listdir(path)
file_list.sort()
file_list = file_list[9:10]

for i in range(len(file_list)):
    
    y_axis = path + "/" + file_list[i]
    
    num = file_list[i][:2]
    
    PSnED_graph("Normal", num)

PS velocity 개수 :  1647
ED velocity 개수 :  1648


In [156]:
path = "D:/kist_storage/Ultrasound Imaging Processing/smoothing_yarray"
file_list = os.listdir(path)
file_list = file_list[11:12]
file_list

['13_smoothing_yarray.npy']

## TAMAX

In [115]:
from __future__ import print_function

import numpy as np
from scipy.integrate import simps
from numpy import trapz

In [215]:
def TAMAX(group, num):
    
    f = open("./Ultrasound Imaging Processing_real_update/"+group+"/data_TAMAX/"+num+"_TAMAX.txt", "w")

    x = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/smoothing_xarray/'+num+'_smoothing_xarray.npy')
    y = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/smoothing_yarray/'+num+'_smoothing_yarray.npy')

    # ED만 가지고 계산
    x_axis = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/EDvelocity_xarray/'+num+'_EDvelocity_xarray.npy')
    

    k = 0
    j = 0
    for j in range(len(x_axis)-2):
        area_x = x_axis[j:j+2]
    
        start_index = 0
        end_index = 0
    
        for i in range(len(x)):
            if x[i] == area_x[0]:
                start_index = i
        
            if x[i] == area_x[1]:
                end_index = i
                break
        
        area_y = y[start_index:end_index]
        area = simps(area_y, dx=1)
        TAMAX = area/(end_index-start_index)
        
        f.write(str(TAMAX)+"\n")  
        k += 1
    
    f.close()
    
    print("ED velocity 개수 : ", len(x_axis))
    print("TAMAX 개수 : ", k)

In [216]:
path = './Ultrasound Imaging Processing_real_update/Normal/smoothing_yarray/'
file_list = os.listdir(path)
file_list.sort()
file_list = file_list[9:10]
file_list

['14_smoothing_yarray.npy']

In [217]:
path = './Ultrasound Imaging Processing_real_update/Normal/smoothing_yarray/'
file_list = os.listdir(path)
file_list.sort()
file_list = file_list[9:10]

for i in range(len(file_list)):
    
    y_axis = path + "/" + file_list[i]
    
    num = file_list[i][:2]
    
    TAMAX("Normal", num)

ED velocity 개수 :  1648
TAMAX 개수 :  1646


## Pulsatility Index (PI)

In [119]:
def finding_PI(group, num):
    
    length = 0
    
    tamax = []

    fr = open("./Ultrasound Imaging Processing_real_update/"+group+"/data_TAMAX/"+num+"_TAMAX.txt", 'r')
    lines = fr.readlines()
    for line in lines:
        line = line.strip()
        line = float(line)
        tamax.append(line)
    fr.close()
    
    ed_y_axis = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/EDvelocity_yarray/'+num+'_EDvelocity_yarray.npy')
    ps_y_axis = np.load('./Ultrasound Imaging Processing_real_update/'+group+'/PSvelocity_yarray/'+num+'_PSvelocity_yarray.npy')
    
    if len(ed_y_axis) >= len(ps_y_axis):
        length = len(ps_y_axis)
    else:
        length = len(ed_y_axis)
        
    fw = open("./Ultrasound Imaging Processing_real_update/"+group+"/data_PI/"+num+"_PI.txt", "w")    
    
    k = 0
    for i in range(length):
        
        if len(tamax) <= i:
            break
            
        son = ps_y_axis[i]-ed_y_axis[i]
        pi = son / tamax[i]                    # tamax 개수만큼 출력
        
        fw.write(str(pi)+"\n")  
        
        k += 1
        
    fw.close()
    
    print("PI 개수 : ", k)

In [218]:
path = './Ultrasound Imaging Processing_real_update/Normal/smoothing_yarray/'
file_list = os.listdir(path)
file_list.sort()
file_list = file_list[9:10]

for i in range(len(file_list)):
    
    y_axis = path + "/" + file_list[i]
    
    num = file_list[i][:2]
    
    finding_PI("Normal", num)

PI 개수 :  1646


## AB ratio (Systolic-diastolic ratio)

In [121]:
def finding_ABratio(group, num):
    
    f = open("./Ultrasound Imaging Processing_real_update/"+group+"/data_ABratio/"+num+"_ABratio.txt", "w")

    ed_y = np.load("./Ultrasound Imaging Processing_real_update/"+group+"/EDvelocity_yarray/"+num+'_EDvelocity_yarray.npy')
    ps_y = np.load("./Ultrasound Imaging Processing_real_update/"+group+"/PSvelocity_yarray/"+num+'_PSvelocity_yarray.npy')
    
    
    k = 0
    for i in range(len(ps_y)):
        ab_ratio = ps_y[i] / ed_y[i]
        
        f.write(str(ab_ratio)+"\n")  
        
        k += 1
    
    f.close()
    
    print("AB ratio 개수 : ", k)

In [219]:
path = './Ultrasound Imaging Processing_real_update/Normal/smoothing_yarray/'
file_list = os.listdir(path)
file_list.sort()
file_list = file_list[9:10]

for i in range(len(file_list)):
    
    y_axis = path + "/" + file_list[i]
    
    num = file_list[i][:2]
    
    finding_ABratio("Normal", num)

AB ratio 개수 :  1647


## Resistance Index (RI)

In [123]:
def finding_RI(group, num):
    
    f = open("./Ultrasound Imaging Processing_real_update/"+group+"/data_RI/"+num+"_RI.txt", "w")

    ed_y = np.load("./Ultrasound Imaging Processing_real_update/"+group+"/EDvelocity_yarray/"+num+'_EDvelocity_yarray.npy')
    ps_y = np.load("./Ultrasound Imaging Processing_real_update/"+group+"/PSvelocity_yarray/"+num+'_PSvelocity_yarray.npy')
    
    k = 0
    for i in range(len(ps_y)):
        ri = (ps_y[i]-ed_y[i]) / ps_y[i]
        
        f.write(str(ri)+"\n")  
        
        k += 1
    
    f.close()
    
    print("RI 개수 : ", k)

In [220]:
path = './Ultrasound Imaging Processing_real_update/Normal/smoothing_yarray/'
file_list = os.listdir(path)
file_list.sort()
file_list = file_list[9:10]

for i in range(len(file_list)):
    
    y_axis = path + "/" + file_list[i]
    
    num = file_list[i][:2]
    
    finding_RI("Normal", num)

RI 개수 :  1647
